In [ ]:
import pandas as pd

In [ ]:
my_cols = ["A", "B", "C", "D", "E","A", "B", "C", "D", "E","A", "B", "C", "D", "E"]
df = pd.read_csv("data.csv",
            names=my_cols,
            engine='c')

In [ ]:
df.head()

In [ ]:
df['Datetime'] = pd.to_datetime(df["A"])
df.sort_values(by='Datetime',inplace = True)
#df = df.set_index('Datetime')

In [ ]:
df.dtypes

In [ ]:
df.groupby(['B'])

In [ ]:
imu_df = df.query('B == "imu"')

In [ ]:
imu_df.head()

In [ ]:
# Using plotly.express
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'browser'

fig = px.line(df, x = "Datetime", y =["C","D","E"])
fig.show()